# 这个文件是下载并导入的文件

In [ ]:
import GEOparse
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
GPL96 = GEOparse.get_GEO(geo='GPL96', destdir="./datasets", silent=True)
GPL570 = GEOparse.get_GEO(geo='GPL570', destdir="./datasets", silent=True)
GPL571 = GEOparse.get_GEO(geo='GPL571', destdir="./datasets", silent=True)
GPL2507 = GEOparse.get_GEO(geo='GPL2507', destdir="./datasets", silent=True)

In [ ]:
GSE_IDs = [ "6269",
            "20346",
            "40012",
            # "40396",
            "42026",
            "60244",
            #"66099",
            #"63990",
            "21802",
            "27131",
            "28750",
            "57065",
            #"68310",
            '111368',
            '20346']
noone_key=["Uninfected type 2 diabetes mellitus","Con","non-infectious illness","Vac","Healt","Blood_Control","Control_Healthy","PAXgene whole blood, healthy control","Blood_HV","whole blood-Healthy Control","healthy subject","Uninfected healthy","Uninfected healthy","HC","Vac","pathogen: None","WB-control", "disease: Control"]
bacteria_key=["Inf","TB","NEC","bacterial","bacterial pneumonia","Experiment_Post_Surgical","Experiment_Sepsis","PAXgene whole blood, bacterial pneumonia", "Blood_P","whole blood-BACTERIA","intensive-care unit patient","Septicemic melioidosis","Other sepsis","PBMC_S.aureus_MRSA_INF","disease: Sepsis","disease: SepticShock","PBMC_S.pneumoniae_INF","PBMC_S.aureus_MSSA_INF","bacterial pneumonia_day","pathogen: E.coli","pathogen: Bacteria","pathogen: MRSA","pathogen: Salmonella","pathogen: MSSA","WB-bact",]
virus_key=["infection: influenza A virus","infection: influenza B virus","infection: human coronavirus NL63","infection: human coronavirus HKU1","infection: human rhinovirus","infection: enterovirus","infection: respiratory syncytial virus A","Vir","viral","Severe Influenza","Pande","blood_day","PAXgene whole blood, SIRS","PAXgene whole blood, influenza A pneumonia","whole blood-VIRUS","H1N1","A","B","H3N2","PBMC_InfluenzaA_INF","PBMC_InfluenzaB_INF","Severe Influenza_day","pathogen: Adenovirus","disease: SIRS","pathogen: HHV6","pathogen: Enterovirus","pathogen: Rhinovirus","WB-H1N1","WB-RSV","Adenovirus","pathogen: Adenovirus"]
both_key=["PAXgene whole blood, mixed bacterial and influenza A pneumonia","whole blood-COINFECTION"]
other_key=["infection: our tests did not detect one of the viruses sought","Sarcoid","Pneumonia","Cancer"]

In [ ]:
data = pd.DataFrame(
    columns=['GSE','GSM','GPL','description','value','infection','title']
)
for gseid in GSE_IDs:
        GEONAME="GSE" + gseid
        print(GEONAME)
        gse = GEOparse.get_GEO(geo=GEONAME, destdir="./datasets", silent=True)
        sample_value = gse.pivot_samples('VALUE').T

        gsm_list = []
        n = 0
        no_new_gpl = 1

        for i in gse.gsms:
            n += 1

            # 判断属于哪一类
            metadata = gse.gsms[i].metadata
            found = 0
            
            for value in metadata.values():
                if any(virus in value[0] for virus in both_key):
                    infection = "both"
                    found = 1
                    title = value[0]
                    break
                elif any(virus in value[0] for virus in bacteria_key):
                    infection = "bacteria"
                    if found != 0 and found !=2:
                        print('2')
                    found = 2
                    title = value[0]
                    break
                elif any(virus in value[0] for virus in virus_key):
                    infection = "viral"
                    if found != 0 and found !=3:
                        print('3')
                    found = 3
                    title = value[0]
                    break
                elif any(virus in value[0] for virus in other_key):
                    infection = "other"
                    if found != 0 and found !=4:
                        print('4')
                    found = 4
                    title = value[0]
                    break
                elif any(virus in value[0] for virus in noone_key):
                    infection = "noone"
                    if found != 0 and found !=5:
                        print('5')
                    found = 5
                    title = value[0]
                    break
                else:
                    infection = "unknown"
                    print("都找不到")
                    print(metadata)
                    index = 0
                    title = value[0]

                
            
            # 数据处理
            sample_gpl = gse.gsms[i].metadata['platform_id']
            description = gse.gsms[i].columns["description"][1]
            gsm_list.append(sample_gpl)
            if description == "Average normalized":
                value = sample_value.iloc[n-1]
            elif description == "Log2 normalized signal":
                value = 2**sample_value.iloc[n-1]
            elif description == "MAS5-calculated Signal intensity":
                value = sample_value.iloc[n-1]
            elif description == "Normalized value from BeadStudio software":
                value = sample_value.iloc[n-1]
            elif description == "cubic spline normalised intensity":
                value = sample_value.iloc[n-1]
            elif description == "GenomeStudio quantile normalised average signal intensity":
                value = sample_value.iloc[n-1]
            elif description == "quantile-normalized signal intensity":
                value = sample_value.iloc[n-1]
            elif description == "Illumina calculated signal intensity [AVG_Signal]":
                value = sample_value.iloc[n-1]
            elif description == "RMA-normalised signal intensity from Partek Genomics Suite":
                value = 2**sample_value.iloc[n-1]
            elif description == "RMA signal intensity (log base 2)":
                value = 2**sample_value.iloc[n-1]
            elif description == "RMA":
                value = 2**sample_value.iloc[n-1]
            else:
                print(gse.gsms[i].columns["description"][1])
                print(sample_gpl)
                print('Now just use the raw value')
                print('Samples: ', sample_value.iloc[n-1].head())
                print('-'*50)
                value = sample_value.iloc[n-1]

            data = data.append({
                'GSE' : GEONAME,
                'GSM' : metadata['geo_accession'],
                'GPL' : sample_gpl,
                'description' : gse.gsms[i].columns["description"][1],
                'value' : value,
                'infection' : infection,
                'title' : title},ignore_index=True)
data
                

In [ ]:
databygpl = data.groupby(['GPL'])

In [ ]:
gse = GEOparse.get_GEO(geo=GEONAME, destdir="./datasets")
sample_value = gse.pivot_samples('VALUE').T

infection = []
gsm_list = []
n = 0

value = pd.DataFrame()
for i in gse.gsms:
    n += 1

    # 感染状态的处理
    characteristics_ch1 = gse.gsms[i].metadata['characteristics_ch1']   # the infection_status is in the characteristics_ch1
    infection_status = characteristics_ch1[-1].split(':')[1].strip()            # 但characteristics_ch1有的有两个值，有的有三个，取最后一个
    infection.append(infection_status)

    sample_gpl = gse.gsms[i].metadata['platform_id']
    gsm_list.append(sample_gpl)
    # 对数等的处理 # 可能不同做法
    if sample_gpl == ['GPL96']: #  MAS5-calculated Signal intensityzs
        value = pd.concat([value, sample_value.iloc[n-1]], axis=1)
        #部分样品跑偏严重
        # infection.pop() 
    elif sample_gpl == ['GPL570']: #  MAS5-calculated Signal intensity
        value = pd.concat([value, sample_value.iloc[n-1]], axis=1)
        # 直接代入导致分数分布大量超过1（最大值为1），所以先不用 infection.pop() 
    elif sample_gpl == ['GPL571']:
        value = pd.concat([value, 2**sample_value.iloc[n-1]], axis=1)
    elif sample_gpl == ['GPL2507']: #  Normalized value from BeadStudio software
        value = pd.concat([value, 2**sample_value.iloc[n-1]], axis=1)
        # 名不一样，大量nan
        # infection.pop() # 把刚刚加进去的感染状态也删了
    elif sample_gpl == ['GPL6947']:
        value = pd.concat([value, sample_value.iloc[n-1]], axis=1)
    # GPL6947 cubic spline normalised intensity
    else:
        print(gse.gsms[i].columns["description"][1])
        print(sample_gpl)
        print('Now just use the raw value')
        print('Samples: ', sample_value.iloc[n-1].head())
        print('-'*50)
        value = pd.concat([value, sample_value.iloc[n-1]], axis=1)

value = value.T

# merge the infection status and value
value.insert(0, 'infection_status', infection)
value.insert(0, 'gsm', gsm_list)
print(value.head())
# Save the data and Show the first 5 rows
value.to_csv('./datasets/' + GEONAME + '.csv', index=False)
value.head()

In [ ]:
# Save the data and Show the first 5 rows
value.to_csv('./datasets/' + GEONAME + '.csv', index=False)
value.head()

## 不用了

不清楚什么时候再把这个id转化成通用的Gene Symbol。

可能在这里就转，也可以分析结束了再转。

大概需要10分钟

In [ ]:
# get all Series_platform_id
gpls = gse.metadata['platform_id']
    

In [ ]:
# GPL571 = gse.gpls['GPL571']
# n = 0
# for rows in sample_value:
#     Gene_Symbol = GPL571.table[GPL571.table['ID'] == rows]['Gene Symbol']
#     if Gene_Symbol.empty:
#         Gene_Symbol[0] = rows
#     sample_value.rename(columns={rows: Gene_Symbol.values[0]}, inplace=True)
#     n += 1
#     if n % 100 == 0:
#         print("Finished %d rows of %d at %s" % (n, sample_value.shape[1], datetime.datetime.now()))
# sample_value.tail()

In [ ]:
# gse.gpls['GPL571'].table.columns # GPL571, 也就是这个GEO数据集的平台，的列。
# # 从这个平台可以直接读取Gene Symbol，Target Description等等有用的东西。

In [ ]:
# gse.gpls['GPL571'].table.head() # 前5行